In [552]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import xlwings as xl
import numpy as np
from time import sleep

In [554]:
options = webdriver.ChromeOptions()

In [556]:
driver = webdriver.Chrome(options=options)

In [557]:
driver.implicitly_wait(0.5)

In [569]:
wb = xl.Book('GERAL INI-R - CASA DO ESTUDANTE 29.03.xlsx')
sheet_metamodelo = wb.sheets['DADOS METAMODELO']

In [581]:
matriz_de_parametros = np.array(sheet_metamodelo.range('B4:BD58').value)
matriz_de_parametros = matriz_de_parametros.transpose()


In [583]:
for i in range(0,55):
    print (i)
    driver.get('https://cb3e.sites.ufsc.br/interface?')
    bol_array_inputs = np.array(list(map(lambda x: type(x)!=str,matriz_de_parametros[i,:])))
    input_parametros = matriz_de_parametros[:,bol_array_inputs]
    select_parametros = matriz_de_parametros[:,~bol_array_inputs]




    area_total = input_parametros[i,2]
    driver.find_element(By.XPATH,'//*[@id="city"]/option[40]').click()
    driver.find_element(By.XPATH, '//*[@id="area_apt"]').clear()
    driver.find_element(By.XPATH, '//*[@id="area_apt"]').send_keys(area_total)
    driver.find_element(By.XPATH, '//*[@id="num_apps"]').clear()
    driver.find_element(By.XPATH, '//*[@id="num_apps"]').send_keys('1')
    selects = driver.find_elements(By.TAG_NAME, 'select')

    for j,select in enumerate(selects[1:]):

        state=''

        if (select_parametros[i,j]=='Dormitório')|(select_parametros[i,j]=='Sim'):
            state = '0'
        elif (select_parametros[i,j]=='Sala')|(select_parametros[i,j]=='Não'):
            state = '1'

        elif (select_parametros[i,j]=='Contato com o solo'):
            state = 'solo'
        elif (select_parametros[i,j]=='Entre pavimentos') & (j==1):
            state = 'pavimento'

        elif (select_parametros[i,j]=='Entre pavimentos') & (j==2):
            state = 'pav'
        elif (select_parametros[i,j]=='Exposto ao sol e ao vento'):
            state = 'sol'

        for box in select.find_elements(By.TAG_NAME, 'option'):
            if box.get_attribute('value')==state:
                box.click()

    inputs = driver.find_elements(By.TAG_NAME, 'input')
    inputs = inputs[2:]

    for j,input_ in enumerate(inputs):
        keys = input_parametros[i,j]
        
        if keys != None:
            input_.clear()
            input_.send_keys( keys if (j!=10)&(j!=13) else (keys/100))
            sleep(0.00002)
            

    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="btn_calcular"]')))

    try:
        element.click()
    except Exception:
        sleep(0.5)
        element.click()

    sleep(1)

    resultado_metamodelo = []

    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="PHFT"]')))

    resultado_metamodelo.append(float(element.text))
    resultado_metamodelo.append(float(driver.find_element(By.XPATH,'//*[@id="PHsFT"]').text))
    resultado_metamodelo.append(float(driver.find_element(By.XPATH, '//*[@id="PHiFT"]').text))
    resultado_metamodelo.append(float(driver.find_element(By.XPATH, '//*[@id="CgTR"]').text))
    resultado_metamodelo.append(float(driver.find_element(By.XPATH, '//*[@id="CgTA"]').text))
    resultado_metamodelo.append(float(driver.find_element(By.XPATH, '//*[@id="TOMax"]').text))
    resultado_metamodelo.append(float(driver.find_element(By.XPATH, '//*[@id="TOMin"]').text))
    resultado_metamodelo.append(float(driver.find_element(By.XPATH, '//*[@id="Consumo_resfriamento"]').text))
    resultado_metamodelo.append(float(driver.find_element(By.XPATH, '//*[@id="Consumo_aquecimento"]').text))

    resultado_metamodelo = np.array(resultado_metamodelo)

    sheet_metamodelo.range((61,2+i),(69,2+i)).value = resultado_metamodelo.reshape(-1,1)

    tds = driver.find_elements(By.TAG_NAME, 'a')
    tds[-1].find_element(By.TAG_NAME, 'span').click() #avança para a proxima etapa

    dados_continuação_metamodelo = sheet_metamodelo.range((72,2+0),(76,2+0)).value
    opcoes = driver.find_element(By.TAG_NAME, 'select').find_elements(By.TAG_NAME,'option')

    if dados_continuação_metamodelo[0]=='Sem acumulação':
        opcoes[1].click()
    else:
        opcoes[0].click()

    for value,input_ in zip(dados_continuação_metamodelo[1:],driver.find_elements(By.TAG_NAME,'input')):
        input_.send_keys(value)


    driver.find_element(By.XPATH, '/html/body/div[2]/div/div/a[2]/span').click()#resultado da outra janela

    dados_etiquetagem = []
    dados_etiquetagem.append(driver.find_element(By.XPATH, '//*[@id="img_envoltoria"]').get_attribute('src').split('/')[-1].replace('.svg','').replace('Class','')) #pega a classe de efic
    dados_etiquetagem.append(driver.find_element(By.XPATH, '//*[@id="img_UH"]').get_attribute('src').split('/')[-1].replace('.svg','').replace('Class','')) #pega a classe de efic
    dados_etiquetagem.append(driver.find_element(By.XPATH, '//*[@id="img_aq"]').get_attribute('src').split('/')[-1].replace('.svg','').replace('Class','')) #pega a classe de efic
    dados_etiquetagem = np.array(dados_etiquetagem)
    sheet_metamodelo.range((90,2+i),(92,2+i)).value = dados_etiquetagem.reshape(-1,1)

    linhas_de_texto = driver.find_element(By.XPATH, '/html/body/div[5]').find_elements(By.TAG_NAME,'span')
    detalhes_do_consumo = np.array([float(element.text.split(':')[-1].strip().split(' ')[0].replace('%','')) for element in linhas_de_texto])
    detalhes_do_consumo[-2:] = detalhes_do_consumo[-2:]/100
    sheet_metamodelo.range((79,2+i),(87,2+i)).value = detalhes_do_consumo.reshape(-1,1)

29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
